**Dataset**

In these exercises, we’re going to experiment with the Chinook sample DB while using SQLAlchemy module

First, run the code below to download the database locally

In [3]:
### download and extract chinook sample DB
import urllib.request
import zipfile
from functools import partial
import os

chinook_url = 'http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip'
file_name = 'chinook.zip'

if not os.path.exists(file_name):
    print('downloading chinook.zip ', end='')

    # Set up the request with a User-Agent header
    req = urllib.request.Request(chinook_url, headers={'User-Agent': 'Mozilla/5.0'})

    with urllib.request.urlopen(req) as response:
        with open(file_name, 'wb') as f:
            for data in iter(partial(response.read, 4 * 1024), b''):
                print('.', end='', flush=True)
                f.write(data)

# Extract the zip file
zipfile.ZipFile(file_name).extractall()
assert os.path.exists('chinook.db')

The helper methods below will help, you may use for the following exercises :

In [14]:
### useful: functions for displaying results from sql queries using pandas
from IPython.display import display
import pandas as pd

def sql(query):
    print()
    print(query)
    print()

def get_results(query):
    global engine
    q = query.statement if isinstance(query, sqlalchemy.orm.query.Query) else query
    return pd.read_sql(q, engine)

def display_results(query):
    df = get_results(query)
    display(df)
    sql(query)

**🌟 Exercise 1 : Open the database**

open the database using sqlalchemy module interface. create an engine object in a variable named engine

call the connect() method to obtain a connection and place in a variable named cur
now run the code below to to run reflecton on the database, prepare classes that map to the database and create an orm session :

In [6]:
import sqlalchemy

# create an engine
engine = sqlalchemy.create_engine('sqlite:///chinook.db')

# create a connection
cur = engine.connect()

# reflect the database
metadata = sqlalchemy.MetaData()
metadata.reflect(engine)

from sqlalchemy.ext.automap import automap_base
Base = automap_base(metadata=metadata)
Base.prepare()

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

**🌟 Exercise 2 : table names**

print out all the table names

In [7]:
print(metadata.tables.keys())

dict_keys(['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'tracks', 'media_types', 'invoices', 'playlist_track', 'playlists'])


**🌟 Exercise 3 : Tracks**

print out the first three tracks in the tracks table

In [15]:
# map the tracks table correctly
Track = Base.classes.tracks

# query the first three tracks
first_three_tracks = session.query(Track).limit(3)
display_results(first_three_tracks)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99



SELECT tracks."TrackId" AS "tracks_TrackId", tracks."Name" AS "tracks_Name", tracks."AlbumId" AS "tracks_AlbumId", tracks."MediaTypeId" AS "tracks_MediaTypeId", tracks."GenreId" AS "tracks_GenreId", tracks."Composer" AS "tracks_Composer", tracks."Milliseconds" AS "tracks_Milliseconds", tracks."Bytes" AS "tracks_Bytes", tracks."UnitPrice" AS "tracks_UnitPrice" 
FROM tracks
 LIMIT ? OFFSET ?



**🌟 Exercise 4 : Albums from Tracks**

print out the track name and albums title of the first 20 tracks in the tracks table


In [19]:
# Map the albums table correctly
Album = Base.classes.albums

# Join tracks and albums
tracks_albums = session.query(Track.Name, Album.Title).join(Album, Track.AlbumId == Album.AlbumId).limit(20)
display_results(tracks_albums)

,Name,Title
0,For Those About To Rock (We Salute You),For Those About To Rock We Salute You
1,Put The Finger On You,For Those About To Rock We Salute You
2,Let's Get It Up,For Those About To Rock We Salute You
3,Inject The Venom,For Those About To Rock We Salute You
4,Snowballed,For Those About To Rock We Salute You
5,Evil Walks,For Those About To Rock We Salute You
6,C.O.D.,For Those About To Rock We Salute You
7,Breaking The Rules,For Those About To Rock We Salute You
8,Night Of The Long Knives,For Those About To Rock We Salute You
9,Spellbound,For Those About To Rock We Salute You



SELECT tracks."Name" AS "tracks_Name", albums."Title" AS "albums_Title" 
FROM tracks JOIN albums ON tracks."AlbumId" = albums."AlbumId"
 LIMIT ? OFFSET ?



**🌟 Exercise 5 : Tracks sold**

print out the first 10 track sales from the invoice_items table

for these first 10 sales, print what are the names of the track sold, and the quantity sold

In [22]:
InvoiceItem = Base.classes.invoice_items  # use lowercase name

# First 10 track sales
first_10_sales = session.query(InvoiceItem).limit(10).all()

# Print track names and quantities
for item in first_10_sales:
    track = session.query(Track).filter(Track.TrackId == item.TrackId).first()
    print(f"Track: {track.Name}, Quantity Sold: {item.Quantity}")

Track: Balls to the Wall, Quantity Sold: 1
Track: Restless and Wild, Quantity Sold: 1
Track: Put The Finger On You, Quantity Sold: 1
Track: Inject The Venom, Quantity Sold: 1
Track: Evil Walks, Quantity Sold: 1
Track: Breaking The Rules, Quantity Sold: 1
Track: Dog Eat Dog, Quantity Sold: 1
Track: Overdose, Quantity Sold: 1
Track: Love In An Elevator, Quantity Sold: 1
Track: Janie's Got A Gun, Quantity Sold: 1


**🌟 Exercise 6 : Top tracks sold**

print the names of top 10 tracks sold, and how many they times they were sold

In [24]:
from sqlalchemy import func

# top 10 tracks by quantity sold
top_tracks = session.query(Track.Name, func.sum(InvoiceItem.Quantity).label('TotalSold')) \
                    .join(InvoiceItem, Track.TrackId == InvoiceItem.TrackId) \
                    .group_by(Track.Name) \
                    .order_by(func.sum(InvoiceItem.Quantity).desc()) \
                    .limit(10)
display_results(top_tracks)

,Name,TotalSold
0,The Trooper,5
1,Untitled,4
2,The Number Of The Beast,4
3,Sure Know Something,4
4,Hallowed Be Thy Name,4
5,Eruption,4
6,Where Eagles Dare,3
7,Welcome Home (Sanitarium),3
8,Sweetest Thing,3
9,Surrender,3



SELECT tracks."Name" AS "tracks_Name", sum(invoice_items."Quantity") AS "TotalSold" 
FROM tracks JOIN invoice_items ON tracks."TrackId" = invoice_items."TrackId" GROUP BY tracks."Name" ORDER BY sum(invoice_items."Quantity") DESC
 LIMIT ? OFFSET ?



**🌟 Exercise 7 : Top selling artists**

Who are the top 10 highest selling artists?

In [26]:
Artist = Base.classes.artists

# joining Track > Album > Artist > InvoiceItem
top_artists = session.query(Artist.Name, func.sum(InvoiceItem.Quantity).label('TotalSold')) \
                      .join(Album, Album.ArtistId == Artist.ArtistId) \
                      .join(Track, Track.AlbumId == Album.AlbumId) \
                      .join(InvoiceItem, InvoiceItem.TrackId == Track.TrackId) \
                      .group_by(Artist.Name) \
                      .order_by(func.sum(InvoiceItem.Quantity).desc()) \
                     .limit(10)
display_results(top_artists)

,Name,TotalSold
0,Iron Maiden,140
1,U2,107
2,Metallica,91
3,Led Zeppelin,87
4,Os Paralamas Do Sucesso,45
5,Deep Purple,44
6,Faith No More,42
7,Lost,41
8,Eric Clapton,40
9,R.E.M.,39



SELECT artists."Name" AS "artists_Name", sum(invoice_items."Quantity") AS "TotalSold" 
FROM artists JOIN albums ON albums."ArtistId" = artists."ArtistId" JOIN tracks ON tracks."AlbumId" = albums."AlbumId" JOIN invoice_items ON invoice_items."TrackId" = tracks."TrackId" GROUP BY artists."Name" ORDER BY sum(invoice_items."Quantity") DESC
 LIMIT ? OFFSET ?

